In [41]:
from pptx import Presentation

In [42]:
from pptx.util import Inches, Pt
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT
from pptx.dml.color import RGBColor

In [43]:
import json

with open("content.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print(data)


{'slides': [{'id': 1, 'type': 'title', 'title': 'Navigating the Classroom Continuum: Online vs. Offline Learning', 'bullets': [], 'notes': "Good morning everyone. Today, we'll critically examine the evolving landscape of higher education, focusing on online and offline learning modalities. Our goal is to empower modern students to make informed decisions about their academic paths.", 'image_keywords': ['student thinking', 'online learning', 'campus life'], 'design_aesthetics': {'layout': 'full-bleed-image', 'title_position': 'center', 'text_style': 'highlighted', 'image_style': 'background'}, 'image_path': 'temp_images\\temp_image_slide1.jpg'}, {'id': 2, 'type': 'content', 'title': "Our Learning Journey: Today's Agenda", 'bullets': ["Unpack higher education's evolving landscape.", 'Deconstruct online learning advantages.', 'Explore traditional classroom strengths.', 'Identify key distinctions and considerations.', 'Conclude with optimal learning environment selection.'], 'notes': "We'l

In [44]:
prs = Presentation()

All other slides

In [45]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT
from pptx.enum.shapes import MSO_SHAPE
from pptx.dml.color import RGBColor
import os


In [46]:
def add_overlay(slide, left, top, width, height, color_overlay):
    """Adds a black/brand color semi-transparent rectangle behind text."""
    rect = slide.shapes.add_shape(
        MSO_SHAPE.RECTANGLE, left, top, width, height
    )
    fill = rect.fill
    fill.solid()
    fill.fore_color.rgb = RGBColor(
        color_overlay["r"],
        color_overlay["g"],
        color_overlay["b"]
    )
    fill.transparency = color_overlay["alpha"]
    rect.line.fill.background()


def center_position(prs, box_w, box_h):
    left = (prs.slide_width - box_w) / 2
    top = (prs.slide_height - box_h) / 2
    return left, top


In [ ]:

from pptx.util import Inches, Pt
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT
from pptx.dml.color import RGBColor
from pptx.enum.shapes import MSO_SHAPE
from PIL import Image
import os


BACKGROUND_COLOR = RGBColor(22, 22, 28)       
CONTENT_BG = RGBColor(255, 255, 255)          
ACCENT = RGBColor(96, 50, 150)                
TEXT_COLOR = RGBColor(30, 30, 30)         
MUTED = RGBColor(110, 110, 110)               

MARGIN = Inches(0.6)
CONTENT_W = prs.slide_width - 2 * MARGIN
CONTENT_H = prs.slide_height - 2 * MARGIN
MAX_IMG_W = int(CONTENT_W * 0.42)
MAX_IMG_H = int(CONTENT_H - Inches(1))


def sanitize_text(text):
    if not isinstance(text, str):
        return text
    
    cleaned = text.replace('***', ' ').replace('**', ' ').replace('*', ' ')
    
    return ' '.join(cleaned.split())


def set_slide_background_dark(slide):
    bg = slide.background
    fill = bg.fill
    fill.solid()
    fill.fore_color.rgb = BACKGROUND_COLOR


def add_content_panel(slide):
    panel = slide.shapes.add_shape(
        MSO_SHAPE.ROUNDED_RECTANGLE,
        MARGIN,
        MARGIN,
        CONTENT_W,
        CONTENT_H
    )
    panel.fill.solid()
    panel.fill.fore_color.rgb = CONTENT_BG
    
    line = panel.line
    line.fill.solid()
    try:
        line.color.rgb = ACCENT
    except Exception:
        
        panel.line.fill.fore_color.rgb = ACCENT
    try:
        line.width = Pt(1.8)
    except Exception:
        pass
    return panel


def fit_image_size(img_path, max_w, max_h):
    with Image.open(img_path) as im:
        w_px, h_px = im.size
        dpi = im.info.get('dpi', (72, 72))[0] or 72
        # convert pixels to inches
        w_in = w_px / dpi
        h_in = h_px / dpi
        img_w = Inches(w_in)
        img_h = Inches(h_in)
        scale = min(max_w / img_w, max_h / img_h, 1)
        return int(img_w * scale), int(img_h * scale)

def place_image_inside(slide, img_path, left, top, max_w, max_h):
    w, h = fit_image_size(img_path, max_w, max_h)
    slide.shapes.add_picture(img_path, left, top, width=w, height=h)


def create_text_box(slide, left, top, width, height, text, size=Pt(18), bold=False, color=TEXT_COLOR, align=PP_PARAGRAPH_ALIGNMENT.LEFT):
    text = sanitize_text(text)
    tf = slide.shapes.add_textbox(left, top, width, height).text_frame
    tf.word_wrap = True
   
    try:
        tf.margin_top = int(Inches(0.06))
        tf.margin_left = int(Inches(0.06))
        tf.margin_right = int(Inches(0.06))
        tf.margin_bottom = int(Inches(0.06))
    except Exception:
        pass
    tf.clear()
    p = tf.paragraphs[0]
    p.text = text
    p.font.size = size
    p.font.bold = bold
    p.font.color.rgb = color
    p.alignment = align
    return tf


def add_footer(slide, text=''):
    if not text:
        text = 'Generated with Powerpoint Generator Agent'
    w = Inches(3)
    h = Inches(0.28)
    left = prs.slide_width - w - MARGIN
    top = prs.slide_height - h - Inches(0.18)
    tf = slide.shapes.add_textbox(left, top, w, h).text_frame
    tf.word_wrap = False
    tf.clear()
    p = tf.paragraphs[0]
    p.text = sanitize_text(text)
    p.font.size = Pt(9)
    p.font.color.rgb = MUTED
    p.alignment = PP_PARAGRAPH_ALIGNMENT.RIGHT



def gen_title(slide_json, prs):
    slide = prs.slides.add_slide(prs.slide_layouts[6])
    set_slide_background_dark(slide)
    add_content_panel(slide)

    img = slide_json.get('image_path')
    if img and os.path.exists(img):
        left = MARGIN + CONTENT_W - MAX_IMG_W - Inches(0.25)
        top = MARGIN + Inches(0.6)
        place_image_inside(slide, img, left, top, MAX_IMG_W, MAX_IMG_H)

  
    tb_left = MARGIN + Inches(0.8)
    tb_top = MARGIN + Inches(0.9)
    tb_w = CONTENT_W - MAX_IMG_W - Inches(1.8)
    tb_h = Inches(1.8)
    create_text_box(slide, tb_left, tb_top, tb_w, tb_h, slide_json.get('title',''), size=Pt(42), bold=True, color=TEXT_COLOR)

    
    subtitle = slide_json.get('subtitle','')
    if subtitle:
        create_text_box(slide, tb_left, tb_top + Inches(1.6), tb_w, Inches(0.9), subtitle, size=Pt(16), color=MUTED)

    add_footer(slide)
    return slide


def gen_full_bleed(slide_json, prs):
    slide = prs.slides.add_slide(prs.slide_layouts[6])
    set_slide_background_dark(slide)
    
    img = slide_json.get('image_path')
    if img and os.path.exists(img):
        left = MARGIN
        top = MARGIN
        slide.shapes.add_picture(img, left, top, width=CONTENT_W, height=CONTENT_H)

    title = slide_json.get('title','')
    if title:
        box_h = Inches(0.9)
        rect = slide.shapes.add_shape(MSO_SHAPE.RECTANGLE, MARGIN + Inches(0.6), MARGIN + CONTENT_H - box_h - Inches(0.25), CONTENT_W - Inches(1.2), box_h)
        rect.fill.solid()
        rect.fill.fore_color.rgb = ACCENT
        rect.fill.transparency = 0.18
        rect.line.fill.background()
        create_text_box(slide, MARGIN + Inches(0.75), MARGIN + CONTENT_H - box_h - Inches(0.15), CONTENT_W - Inches(1.5), box_h, title, size=Pt(26), bold=True, color=RGBColor(255,255,255), align=PP_PARAGRAPH_ALIGNMENT.CENTER)

    add_footer(slide)
    return slide


def gen_text_left_image_right(slide_json, prs):
    slide = prs.slides.add_slide(prs.slide_layouts[6])
    set_slide_background_dark(slide)
    add_content_panel(slide)

    img = slide_json.get('image_path')
    title = sanitize_text(slide_json.get('title',''))
    bullets = [sanitize_text(b) for b in slide_json.get('bullets', [])]

    left = MARGIN + Inches(0.8)
    top = MARGIN + Inches(0.8)
    left_w = int(CONTENT_W * 0.56)
    left_h = CONTENT_H - Inches(1.4)

    
    title_box_h = Inches(1.1)
    create_text_box(slide, left, top, left_w, title_box_h, title, size=Pt(26), bold=True)

    body_top = top + title_box_h + Inches(0.15)
    body_text = '\n'.join(['• ' + b for b in bullets])
    create_text_box(slide, left, body_top, left_w, left_h - title_box_h, body_text, size=Pt(16), color=TEXT_COLOR)

    
    if img and os.path.exists(img):
        img_left = MARGIN + left_w + Inches(0.6)
        img_top = MARGIN + Inches(0.8)
        place_image_inside(slide, img, img_left, img_top, int(CONTENT_W - left_w - Inches(1.2)), left_h)

    add_footer(slide)
    return slide


def gen_image_left_text_right(slide_json, prs):
    slide = prs.slides.add_slide(prs.slide_layouts[6])
    set_slide_background_dark(slide)
    add_content_panel(slide)

    img = slide_json.get('image_path')
    title = sanitize_text(slide_json.get('title',''))
    bullets = [sanitize_text(b) for b in slide_json.get('bullets', [])]

    if img and os.path.exists(img):
        img_left = MARGIN + Inches(0.8)
        img_top = MARGIN + Inches(0.8)
        place_image_inside(slide, img, img_left, img_top, MAX_IMG_W, MAX_IMG_H)

    tx_left = MARGIN + MAX_IMG_W + Inches(1.2)
    tx_top = MARGIN + Inches(0.8)
    tx_w = CONTENT_W - MAX_IMG_W - Inches(1.8)
    tx_h = CONTENT_H - Inches(1.4)

    title_box_h = Inches(0.9)
    create_text_box(slide, tx_left, tx_top, tx_w, title_box_h, title, size=Pt(26), bold=True)
    create_text_box(slide, tx_left, tx_top + title_box_h + Inches(0.1), tx_w, tx_h - title_box_h, '\n'.join(['• ' + b for b in bullets]), size=Pt(16))

    add_footer(slide)
    return slide


def gen_two_column(slide_json, prs):
    slide = prs.slides.add_slide(prs.slide_layouts[6])
    set_slide_background_dark(slide)
    add_content_panel(slide)

    title = sanitize_text(slide_json.get('title',''))
    bullets = [sanitize_text(b) for b in slide_json.get('bullets', [])]
    left_col = bullets[:len(bullets)//2]
    right_col = bullets[len(bullets)//2:]

    create_text_box(slide, MARGIN + Inches(0.8), MARGIN + Inches(0.6), CONTENT_W, Inches(0.9), title, size=Pt(26), bold=True)
    create_text_box(slide, MARGIN + Inches(0.8), MARGIN + Inches(1.6), int(CONTENT_W*0.45), CONTENT_H - Inches(1.6), '\n'.join(['• ' + t for t in left_col]), size=Pt(16))
    create_text_box(slide, MARGIN + int(CONTENT_W*0.52), MARGIN + Inches(1.6), int(CONTENT_W*0.45), CONTENT_H - Inches(1.6), '\n'.join(['• ' + t for t in right_col]), size=Pt(16))

    add_footer(slide)
    return slide


def gen_quote(slide_json, prs):
    slide = prs.slides.add_slide(prs.slide_layouts[6])
    set_slide_background_dark(slide)
    add_content_panel(slide)

    quote = sanitize_text((slide_json.get('bullets') or [slide_json.get('title','')])[0])
    box_w = int(CONTENT_W * 0.9)
    box_h = Inches(1.8)
    left = MARGIN + (CONTENT_W - box_w) / 2
    top = MARGIN + (CONTENT_H - box_h) / 2
    rect = slide.shapes.add_shape(MSO_SHAPE.ROUNDED_RECTANGLE, left, top, box_w, box_h)
    rect.fill.solid()
    rect.fill.fore_color.rgb = ACCENT
    rect.fill.transparency = 0.08
    rect.line.fill.background()
    create_text_box(slide, left + Inches(0.25), top + Inches(0.12), box_w - Inches(0.5), box_h - Inches(0.3), '“' + quote + '”', size=Pt(22), bold=True, color=RGBColor(255,255,255), align=PP_PARAGRAPH_ALIGNMENT.CENTER)

    add_footer(slide)
    return slide

LAYOUT_MAP = {
    'title': gen_title,
    'full-bleed-image': gen_full_bleed,
    'text-left-image-right': gen_text_left_image_right,
    'image-left-text-right': gen_image_left_text_right,
    'two-column': gen_two_column,
    'comparison-split': gen_two_column,
    'quote-emphasis': gen_quote
}

def render_slide(slide_json, prs):
    layout = slide_json.get('design_aesthetics', {}).get('layout', '')
    func = LAYOUT_MAP.get(layout, gen_text_left_image_right)
    return func(slide_json, prs)


for slide_json in data['slides']:
    render_slide(slide_json, prs)

prs.save('final_output.pptx')
